<a href="https://colab.research.google.com/github/snassimr/ApplicationsBuildWithLLMs/blob/main/deoptima_generation_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [1]:
%reload_ext autoreload
%autoreload 2

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount = True)

Mounted at /content/gdrive


In [3]:
SYS_PROJECT_DIR  = '/content/gdrive/MyDrive/Colab Notebooks/deoptima'
SYS_INPUT_DIR    = '/content/gdrive/MyDrive/Colab Notebooks/deoptima/input'
SYS_MODELING_DIR = '/content/gdrive/MyDrive/Colab Notebooks/deoptima/modeling'
SYS_TESTING_DIR = '/content/gdrive/MyDrive/Colab Notebooks/deoptima/testing'
SYS_LLM_MODEL    = 'Mistral-7B-Instruct-v0.3'
SYS_CHAT_IMAGES  = '/content/gdrive/MyDrive/Colab Notebooks/deoptima/images'

In [4]:
import os
import pandas as pd
import shutil

pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
pd.options.mode.copy_on_write = True

# Installs

In [5]:
import os

requirements = """
torch==2.3.1
accelerate==0.32.1
transformers==4.42.4
vllm==0.5.4
gradio==4.41.0
nemoguardrails==0.9.1.1
langchain_community==0.2.11
langchain-openai==0.1.21
"""
requirements = """
gradio==4.41.0
nemoguardrails==0.9.1.1
langchain-openai==0.1.21
"""

requirements_path = os.path.join(SYS_PROJECT_DIR, 'requirements.txt')
with open(requirements_path, 'w') as f:
    f.write(requirements)

65

In [6]:
# import locale
# locale.getpreferredencoding = lambda: "UTF-8"

In [7]:
import os
requirements_path = os.path.join(SYS_PROJECT_DIR, 'requirements.txt')
!pip install -q -r '{requirements_path}'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.5/647.5 kB 33.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.0/303.0 kB 22.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
Reason for being yanked: Regression. AzureChatOpenAI json-mode broken. Fixed in 0.1.22.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 107.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 83.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.8/49.8 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.5/93.5 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.6/55.6 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [8]:
import sys
import os

deoptima_path = os.path.join(SYS_PROJECT_DIR)
if deoptima_path not in sys.path:
    sys.path.append(deoptima_path)

from deoptima.utils import set_logger
from deoptima.utils import dictionary_to_string
from deoptima.utils import conversation_to_string
from deoptima.utils import process_state_to_string

deoptima_logger = set_logger(log_mode = 'S')

/usr/local/lib/python3.10/dist-packages/PIL/Image.py:122: RuntimeWarning: The _imaging extension was built for another version of Pillow or PIL:
Core version: 9.4.0
Pillow version: 10.4.0
  warnings.warn(str(v), RuntimeWarning)
[autoreload of PIL.Image failed: Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/usr/local/lib/python3.10/dist-packages/IPython/extensions/autoreload.py", line 394, in superreload
    module = reload(module)
  File "/usr/lib/python3.10/imp.py", line 315, in reload
    return importlib.reload(module)
  File "/usr/lib/python3.10/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 619, in _exec
  File "<frozen importlib._bootstrap_external>", line 883, in exec_module
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_removed
  File "/

In [9]:
import os

from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
openai_api_key = userdata.get("OPENAI_API_KEY")

# Evaluation

## Conversation

In [15]:
scenario_code = "S0"
# scenario_code = "S1"
n_version = 1

In [16]:
import os

conversation_output_folder = os.path.join(SYS_TESTING_DIR, "output", scenario_code)
conversation_files = [f for f in os.listdir(conversation_output_folder) if os.path.isfile(os.path.join(conversation_output_folder, f))]
conversation_files = [file for file in conversation_files if file.split("_")[1] == str(n_version)]
conversation_files

['abc_1_1.json']

In [18]:
import os
import json
def load_from_json(filepath):
    with open(filepath, 'r') as file:
        return json.load(file)

conversation_idx = 0
conversation = load_from_json(os.path.join(conversation_output_folder, conversation_files[conversation_idx]))

In [19]:
from IPython.display import display, HTML
from deoptima.utils import format_conversation

html_content = format_conversation(conversation)
display(HTML(html_content))

## get_evaluation_data

In [58]:
import sys
import os

deoptima_path = os.path.join(SYS_PROJECT_DIR)
if deoptima_path not in sys.path:
    sys.path.append(deoptima_path)

from deoptima.utils import conversation_to_string
from deoptima.utils import conversation_to_string_numbered

In [59]:
import os
import yaml

with open(os.path.join(deoptima_path, 'deoptima', 'generation_evaluation_data.yaml'), 'r') as file:
    generation_evaluation_data = yaml.safe_load(file)

with open(os.path.join(deoptima_path, 'deoptima', 'generation_data.yaml'), 'r') as file:
    generation_data = yaml.safe_load(file)

import os
import json

def load_from_json(filepath):
    with open(filepath, 'r') as file:
        return json.load(file)

generation_learning_examples = load_from_json(os.path.join(os.path.join(SYS_PROJECT_DIR, "deoptima", "generation_learning_examples.json")))


In [64]:
criteria_rating_list = generation_data['infos']['criteria_rating_list']
options_criteria_rating_list = generation_data['infos']['options_criteria_rating_list']
process_technical_explanation = generation_data['infos']['process_technical_explanation']
process_technical_explanation = process_technical_explanation.format(
    criteria_rating_list=criteria_rating_list)

get_evaluation_data_prompt = generation_evaluation_data['prompts']['get_evaluation_data']['prompt']
get_evaluation_data_prompt = get_evaluation_data_prompt.format(
                                                           conversation = conversation_to_string_numbered(conversation),
                                                           process_technical_explanation=process_technical_explanation,
                                                           Assistant_specificity_examples = generation_learning_examples['Assistant_specificity'],
                                                           Assistant_technicality_examples = generation_learning_examples['Assistant_technicality'])

In [66]:
# from IPython.display import Markdown
# display(Markdown(f"{get_evaluation_data_prompt}"))

In [67]:
from typing import Optional, Literal, List, Dict, Tuple

def get_evaluation_data(prompt : str) -> Tuple[Dict[Literal['assistant'], str],List[str]]:
  import os
  from langchain_openai import ChatOpenAI
  from pydantic import BaseModel, Field
  from typing import List, Literal
  from langchain_core.prompts import ChatPromptTemplate
  from langchain_core.prompts import PromptTemplate
  from langchain_core.output_parsers import StrOutputParser
  from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
  from langchain_core.utils.function_calling import convert_to_openai_function

  model = ChatOpenAI(model="gpt-4o", temperature = 0.0, top_p = 1.0)

  output_parser = StrOutputParser()

  prompt = PromptTemplate(
    template=prompt,
  )

  chain = prompt | model | output_parser

  generation = chain.invoke({'prompt' : prompt})

  print(generation)

  import json

  start_string = "```json" ; end_string = "```"
  start_index = generation.find(start_string) + len(start_string)
  end_index = generation.rfind(end_string) - 1

  if start_index != -1 and end_index != -1:
      json_string = generation[start_index:end_index]
      response_data = json.loads(json_string)
  else:
      print("Start or end string not found")

  return response_data


In [68]:
response_data = get_evaluation_data(get_evaluation_data_prompt)

INFO     : 2024-09-05 11:45:07,176 : HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


```json
{
  "metric": "Completeness",
  "score": 1
}
```

### Explanation:

- **Metric: Completeness**
  - **Score = 1**: The last assistant message contains information about the possibility to proceed to Step 3 or Options Evaluation. The last message is: "All basic definitions and criteria comparisons are set. You can now proceed to Options Evaluation."
  - **Score = 0**: This condition is not met because the last assistant message does contain information about proceeding to Step 3.
  - **Score = -1**: This condition is not met because the assistant does provide information about proceeding to Step 3 in the last message.

### Smoothness Calculation:

1. **Message 0 (user)**: "I'm ready to start the decision-making process."
   - **Smooth**: Yes, it initiates the conversation.
2. **Message 1 (assistant)**: "Great! Let's start by defining the problem you're trying to solve. What is the goal you want to achieve? Please provide a brief description of the problem. After that, we can move

JSONDecodeError: Extra data: line 6 column 1 (char 46)

In [306]:
import time
start_time = time.time()

try:
  response_data = get_evaluation_data(get_evaluation_data_prompt)
except Exception as e:
  deoptima_logger.error(f"Error in {get_evaluation_data.__name__}: {str(e)}")
else:
  deoptima_logger.info(f"{get_evaluation_data.__name__} finished: {str(response_data)}")


end_time = time.time()
runtime_ms = (end_time - start_time) * 1000
print(f"Function runtime: {runtime_ms:.2f} msec")

INFO     : 2024-09-05 00:03:50,072 : HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO     : 2024-09-05 00:03:50,077 : get_evaluation_data finished: [{'metric': 'Completeness', 'score': 1}, {'metric': 'Smoothness', 'score': 100}, {'metric': 'User_specificity', 'score': 100}, {'metric': 'Assistant_specificity', 'score': 83.33}]


```json
[
  {
    "metric": "Completeness",
    "score": 1
  },
  {
    "metric": "Smoothness",
    "score": 100
  },
  {
    "metric": "User_specificity",
    "score": 100
  },
  {
    "metric": "Assistant_specificity",
    "score": 83.33
  }
]
```
Function runtime: 1115.00 msec


# generate_evaluation_data

In [256]:
# scenario_code = "S0"
scenario_code = "S1"

In [257]:
import os

conversation_output_folder = os.path.join(SYS_TESTING_DIR, "output", scenario_code)
conversation_files = [f for f in os.listdir(conversation_output_folder) if os.path.isfile(os.path.join(conversation_output_folder, f))]

In [ ]:
import os
import json
def load_from_json(filepath):
    with open(filepath, 'r') as file:
        return json.load(file)

conversation_idx = 1
conversation = load_from_json(os.path.join(conversation_output_folder, conversation_files[conversation_idx]))

In [ ]:
# from IPython.display import display, HTML
# from deoptima.utils import format_conversation

# html_content = format_conversation(conversation)
# display(HTML(html_content))

In [ ]:
import sys
import os

deoptima_path = os.path.join(SYS_PROJECT_DIR)
if deoptima_path not in sys.path:
    sys.path.append(deoptima_path)

from deoptima.utils import conversation_to_string

import os
import yaml

with open(os.path.join(deoptima_path, 'deoptima', 'generation_evaluation_data.yaml'), 'r') as file:
    generation_evaluation_data = yaml.safe_load(file)

def generate_evaluation_data(conversation: List[Dict[str, str]]):

  ### get_evaluation_data
  get_evaluation_data_prompt = generation_evaluation_data['prompts']['get_evaluation_data']['prompt']
  get_evaluation_data_prompt = get_evaluation_data_prompt.format(process_technical_explanation=process_technical_explanation,
                                                             conversation=conversation_to_string(conversation),
                                                             process_state=process_state_to_string(process_state))

  try:
    response_data = get_evaluation_data(get_evaluation_data_prompt)
    response_finished = response_data['step_finished']
  except Exception as e:
    deoptima_logger.error(f"Error in {get_evaluation_data.__name__}: {str(e)}")
  else:
    deoptima_logger.info(f"{get_evaluation_data.__name__} finished: {response_finished}")

  return response_data


FileNotFoundError: [Errno 2] No such file or directory: '/content/gdrive/MyDrive/Colab Notebooks/deoptima/deoptima/generation_data.yaml'

# Run Evaluation

In [ ]:
import time
start_time = time.time()

# scenario_code = "S0"
scenario_code = "S1"

conversation_extra_length = 10
start_conversation_length = len(conversation)
run_cnt = 1 ; run_limit = 10

while not(stop_criterion):
  deoptima_logger.info(f"{generate_model_response.__name__} started.")
  try:
    response_finished, all_validity_checks_passed, assistant_message,  user_examples = generate_model_response(conversation)
    import random
    user_message = random.choice(user_examples)
  except Exception as e:
    deoptima_logger.error(f"Error in {generate_model_response.__name__}: {str(e)}")
  else:
    deoptima_logger.info(f"assistant_message: {assistant_message}")
    deoptima_logger.info(f"user_message : {user_message}")
    deoptima_logger.info(f"{generate_model_response.__name__} finished.")

    conversation.append({"role": "assistant", "content": assistant_message})
    conversation.append({"role": "user", "content": user_message})
    run_cnt += 1

  stop_criterion = (all_validity_checks_passed) or (len(conversation) > (start_conversation_length + conversation_extra_length)) or run_cnt > run_limit

import os
import json

conversation_output_folder = os.path.join(SYS_TESTING_DIR, "output", scenario_code)

def save_to_json(obj, filepath):
    with open(filepath, 'w') as file:
        json.dump(obj, file, indent=4)

save_to_json(conversation, os.path.join(conversation_output_folder, conversation_files[conversation_idx]))

end_time = time.time()
runtime_ms = (end_time - start_time) * 1000
print(f"Function runtime: {runtime_ms:.2f} msec")


INFO     : 2024-09-04 14:27:53,780 : generate_model_response started.
INFO     : 2024-09-04 14:27:57,230 : HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO     : 2024-09-04 14:27:57,235 : get_process_state finished: 
Problem: select the best laptop

Options:
- Laptop model 1
- Laptop model 2
- Laptop model 3

Criteria:
- Price
- Performance
- Battery Life

Criteria Comparisons:
- Price is Moderately More Important than Performance
- Battery Life is Strongly More Important than Price

INFO     : 2024-09-04 14:27:59,355 : HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO     : 2024-09-04 14:27:59,361 : get_response_data finished: No
INFO     : 2024-09-04 14:27:59,362 : assistant_message: Thank you for the comparisons! Now, please provide the next criteria comparison.
INFO     : 2024-09-04 14:27:59,363 : user_message : Battery Life is Equally Important to Performance.
INFO     : 2024-09-04 14:27:59,365 : generate_model

Function runtime: 28474.07 msec


In [ ]:
from IPython.display import display, HTML
from deoptima.utils import format_conversation

html_content = format_conversation(conversation)
display(HTML(html_content))

# generation_evaluation_data.yaml

In [56]:
generation_evaluation_data = """
prompts:
  get_evaluation_data:
    prompt: |

      Conversation
      ============

      {conversation}

      Process Technical Explanation
      =============================

      {process_technical_explanation}

      Task
      ====

      You are an evaluation system tasked with assessing the quality of conversation consists of assitant and user messages.
      The assistant message starts with "assistant:" , user message starts with "user:".
      The conversation contains user and assistant messages in chronological order from top to bottom.
      The goal of conversation to fill definitions in "Process Technical Explanation".

      - Metric: Completness - how the conversation has been ended.
        -- Score = 1: if last assistant message contains information about possibility to proceed to Step 3 or Options Evaluation.
        -- Score = 0: if any assistant message contains information about possibility to proceed to Step 3 or Options Evaluation , but it's not the last assistant message.
        -- Score = -1: if non of assistant message contains information about possibility to proceed Step 3 or Options Evaluation.
      - Metric: Smoothness - at what degree each user or assistant message follow from previous conversation
        -- Check Each Message: For each message, determine whether it can be perfectly concluded from previous conversation.
        -- Classify each message as either "smooth" or "not smooth".
        -- Smoothness score: Calculate the percentage of messages that are smooth. Formula: (Number of "smooth" messages / Total number of messages) * 100
      - Metric : User_specificity - Evaluates the degree to which user provides specific information on how to fulfill some definition.
        Steps:
        -- Check Each Message: For each user message, determine whether it is specific or not. A specific message provides clear, actionable information.
        -- Classify each message as either "specific" or "not specific".
        -- User_specificity score: Calculate accurately the percentage of user messages that are "specific". Formula: (Number of "specific" user messages / Total number of user messages) * 100
      - Metric: Assistant_specificity - Evaluates the degree to which assistant provides specific information on how to fulfill some definition.
        Steps:
        -- Check Each Message: For each assistant message, determine whether it is specific or not. A specific message provides clear, actionable information.
        -- Classify each message as either "specific" or "not specific".
           Examples:
           {Assistant_specificity_examples}
        -- Assistant_specificity score: Calculate accurately the percentage of assistant messages that are "specific". Formula: (Number of "specific" assistant messages / Total number of assistant messages) * 100
      - Metric : Assistant_technicality - Evaluates the degree to which user provides specific information on how to fulfill some definition.
        Steps:
        -- Check Each Message: For each assistant message, determine whether it contain technical information from "Process Technical Explanation" : Yes or Now. A technical message expose formats, formulas or calculations.
        -- Classify each message as either "technical" or "not technical".
           Examples:
           {Assistant_technicality_examples}
        -- Assistant_technicality score: Calculate accurately the percentage of assistant messages that are "not technical". Formula: (Number of "not technical" assistant messages / Total number of assistant messages) * 100

      Think step-by-step and provide detailed explanation for score calculations.

      Task Output: format the output as a JSON object as follows:
      output:
        metric: ?
        score: ?

 """

In [57]:
# Write the string to a text file
import os
with open(os.path.join(SYS_PROJECT_DIR, "deoptima", "generation_evaluation_data.yaml"), 'w') as file:
    file.write(generation_evaluation_data)

3802

# generation_learning_examples.json

In [54]:
Assistant_specificity_examples_dict = [
{'assistant': "Thank you! Now we have the problem, options, and criteria defined. Please compare each unique pair of criteria. For example, you can say 'Price is Equally Important to Performance'",
'classification': "specific"},
{'assistant': "Thank you for the comparisons! Now, please provide the next criteria comparison.",
'classification': "not specific"},
]

Assistant_technicality_examples_dict = [
{"assistant": "Thank you for your comparison! Now, please provide the remaining comparisons for the other pairs of criteria. Use the format ' is to/than '.",
"classification": "technical"},
{"assistant": "Thank you for your comparison! Now, please provide the remaining comparisons for the other pairs of criteria.",
"classification": "not technical"},
]

template = """<assistant: "{assistant}", classification: "{classification}"/>"""

Assistant_specificity_examples_str = []
for i in Assistant_specificity_examples_dict:
  i_str = template.format(assistant = i['assistant'], classification = i['classification'])
  Assistant_specificity_examples_str.append(i_str)
Assistant_technicality_examples_str = []
for i in Assistant_technicality_examples_dict:
  i_str = template.format(assistant = i['assistant'], classification = i['classification'])
  Assistant_technicality_examples_str.append(i_str)

generation_learning_examples = dict()
generation_learning_examples['Assistant_specificity'] = Assistant_specificity_examples_str
generation_learning_examples['Assistant_technicality'] = Assistant_technicality_examples_str


In [55]:
import os
import json

def save_to_json(obj, filepath):
    with open(filepath, 'w') as file:
        json.dump(obj, file, indent=4)

save_to_json(generation_learning_examples, os.path.join(os.path.join(SYS_PROJECT_DIR, "deoptima", "generation_learning_examples.json")))

# Sleep

In [49]:
import time
time.sleep(30000)

KeyboardInterrupt: 